In [1]:
import mysql.connector
import pandas as pd

Connecting to database

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="my-secret-pw",
  database="droptime"
)

In [3]:
mycursor = mydb.cursor()

quering

In [37]:
query = """
SELECT *, TIMESTAMPDIFF(SECOND, segment_start_time, segment_end_time) AS time_diff_in_seconds 
FROM route_segments
ORDER BY driver_id, segment_id DESC, segment_end_time ASC
"""
df = pd.read_sql(query, mydb)

/tmp/ipykernel_166287/81233910.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


In [38]:
df

,segment_id,driver_id,segment_type,order_id,segment_start_time,segment_end_time,time_diff_in_seconds
0,4973,1,DRIVE,NaN,2024-02-18 18:22:33,2024-02-18 18:28:46,373
1,4972,1,STOP,1950.0,2024-02-18 18:31:46,2024-02-18 18:32:46,60
2,4949,1,STOP,NaN,2024-02-01 05:07:48,2024-02-01 05:19:08,680
3,4948,1,DRIVE,NaN,2024-02-01 05:15:22,2024-02-01 05:24:53,571
4,4947,1,STOP,2080.0,2024-02-01 05:27:53,2024-02-01 05:29:03,70
...,...,...,...,...,...,...,...
4986,10,4,STOP,331.0,2024-02-26 07:03:21,2024-02-26 07:06:16,175
4987,3,4,DRIVE,NaN,2024-02-24 09:15:02,2024-02-24 09:23:31,509
4988,2,4,STOP,1273.0,2024-02-24 09:26:31,2024-02-24 09:32:12,341
4989,1,4,DRIVE,NaN,2024-02-09 14:51:20,2024-02-09 14:55:42,262


In [39]:
# deleting cases when segment type is STOP and don't have order id
df = df.drop(df[(df['segment_type'] == "STOP") & (pd.isna(df['order_id']))].index)

In [40]:
df

,segment_id,driver_id,segment_type,order_id,segment_start_time,segment_end_time,time_diff_in_seconds
0,4973,1,DRIVE,NaN,2024-02-18 18:22:33,2024-02-18 18:28:46,373
1,4972,1,STOP,1950.0,2024-02-18 18:31:46,2024-02-18 18:32:46,60
3,4948,1,DRIVE,NaN,2024-02-01 05:15:22,2024-02-01 05:24:53,571
4,4947,1,STOP,2080.0,2024-02-01 05:27:53,2024-02-01 05:29:03,70
5,4928,1,DRIVE,NaN,2024-02-04 03:55:26,2024-02-04 04:02:30,424
...,...,...,...,...,...,...,...
4986,10,4,STOP,331.0,2024-02-26 07:03:21,2024-02-26 07:06:16,175
4987,3,4,DRIVE,NaN,2024-02-24 09:15:02,2024-02-24 09:23:31,509
4988,2,4,STOP,1273.0,2024-02-24 09:26:31,2024-02-24 09:32:12,341
4989,1,4,DRIVE,NaN,2024-02-09 14:51:20,2024-02-09 14:55:42,262


droping doplicates

In [41]:
df = df.drop_duplicates(subset=["driver_id", "segment_type", "order_id", "segment_start_time", "segment_end_time"], keep="last")

In [42]:
df.to_csv("test4.csv")

chceking if drive => stop sequence

In [47]:
next_row_is_stop = df['segment_type'].shift(-1) == 'STOP'
next_row_is_stop

In [49]:
next_row_is_drive = df['segment_type'].shift(-1) == 'DRIVE'
next_row_is_drive

In [52]:
x = next_row_is_drive + next_row_is_stop

In [54]:
false_indices = x.loc[~x].index

In [55]:
false_indices

Index([4990], dtype='int64')

check if drive is earlier then stop

check if deliver is in the same day?